In [293]:
import pandas as pd
import numpy as np
import altair as alt

In [315]:
# Set max rows to a higher value
alt.data_transformers.disable_max_rows()

# Load the datasets
arrivals_df = pd.read_csv("../data/export/df_cleaned_Inbound_Tourism_Arrivals.csv")
transport_df = pd.read_csv("../data/export/df_cleaned_Inbound_Tourism_Transport.csv")
regions_df = pd.read_csv("../data/export/df_cleaned_Inbound_Tourism_Regions.csv")
currency_df = pd.read_csv('../data/raw/currency_data.csv')

# Preview datasets
arrivals_df.head()


,Country,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ALBANIA,304.0,287.0,119.0,184.0,371.0,317.0,354.0,470.0,557.0,...,3256.0,3673.0,4131.0,4736.0,5118.0,5927.0,6406.0,2658.0,5689.0,7543.8
1,ALGERIA,520.0,605.0,635.0,678.0,749.0,866.0,901.0,988.0,1166.0,...,2733.0,2301.0,1710.0,2039.0,2451.0,2657.0,2371.0,591.0,125.0,1398.0
2,AMERICAN SAMOA,34.0,35.0,26.0,36.0,41.0,44.0,36.0,0.0,0.0,...,49.3,51.6,47.1,38.3,42.3,51.8,58.6,0.9,0.0,0.0
3,ANDORRA,0.0,0.0,0.0,0.0,9422.0,10991.0,11351.0,11507.0,11601.0,...,7676.0,7797.0,7850.0,8025.0,8152.0,8328.0,8235.0,5207.0,5422.0,8426.7
4,ANGOLA,9.0,21.0,45.0,52.0,45.0,51.0,67.0,91.0,107.0,...,650.0,595.0,592.0,397.0,261.0,218.0,218.0,64.0,64.0,130.0


In [316]:
currency_df = currency_df.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68', '2023'], axis=1)
year_columns_currency = [col for col in currency_df.columns if col.isdigit()]
year_columns_filtered = [col for col in year_columns_currency if int(col) >= 1995]

currency_df = currency_df[['Country Name'] + year_columns_filtered]
currency_df.head(10)

,Country Name,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,...,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000,1.790000
1,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,36.567145,47.500000,47.500000,47.500000,46.619531,47.357575,47.500015,47.263000,48.762754,...,55.377500,57.247500,61.143462,67.866086,68.026904,72.083247,77.737949,76.813536,NaN,NaN
3,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,0.002750,0.128029,0.229040,0.392824,2.790706,10.040544,22.057862,43.530207,74.606301,...,96.518279,98.302417,120.060702,163.656434,165.915951,252.855748,364.825805,578.258780,631.441956,460.567512
5,Albania,92.697500,104.498917,148.932917,150.633333,137.690583,143.709417,143.484833,140.154516,121.863250,...,105.669167,105.480000,125.961667,124.142500,119.100000,107.989167,109.850833,108.650000,103.520000,113.041667
6,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.845377,0.950916
7,Arab World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,United Arab Emirates,3.671000,3.671000,3.671125,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500,...,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500,3.672500
9,Argentina,0.999750,0.999663,0.999500,0.999500,0.999500,0.999500,0.999500,3.063257,2.900629,...,5.459353,8.075276,9.233186,14.758175,16.562707,28.094992,48.147892,70.539167,94.990742,130.616550


In [296]:
# Convert currency data to numeric values
currency_df.iloc[:, 1:] = currency_df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
currency_df = currency_df.rename(columns={'Country Name': 'Country'})

# create dfs that use percentage change
df_currency_pct = currency_df.set_index('Country').pct_change(axis=1) * 100
df_visitors_pct = arrivals_df.set_index('Country').pct_change(axis=1) * 100

df_currency_pct.head(10)


/var/folders/y4/kt57fxd942z3zncr4rvv6t740000gn/T/ipykernel_88349/1133129564.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_currency_pct = currency_df.set_index('Country').pct_change(axis=1) * 100


,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,,,,,,,,,,,,,,
Aruba,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,NaN,29.898029,0.000000,0.000000,-1.853619,1.583121,0.300775,-0.498978,3.173209,-1.881438,...,8.750938,3.376823,6.805470,10.994837,0.236964,5.962851,7.844683,-1.189140,0.000000,0.000000
Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,NaN,4555.217420,78.896801,71.508634,610.422376,259.785071,119.687910,97.345542,71.389723,11.976283,...,1.100185,1.848497,22.134028,36.311409,1.380646,52.399903,44.282188,58.502708,9.197124,-27.060990
Albania,NaN,12.731106,42.521015,1.141733,-8.592222,4.371274,-0.156276,-2.321024,-13.050786,-15.659519,...,-2.324739,-0.179018,19.417583,-1.444222,-4.061864,-9.328995,1.723938,-1.093149,-4.721583,9.197901
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.484255
Arab World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United Arab Emirates,NaN,0.000000,0.003405,0.037454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [297]:
df_currency_pct.reset_index(inplace=True)
df_visitors_pct.reset_index(inplace=True)

In [298]:
print(df_currency_pct['Country'].head(10))
print(df_visitors_pct['Country'].head(10))

0                          Aruba
1    Africa Eastern and Southern
2                    Afghanistan
3     Africa Western and Central
4                         Angola
5                        Albania
6                        Andorra
7                     Arab World
8           United Arab Emirates
9                      Argentina
Name: Country, dtype: object
0                ALBANIA
1                ALGERIA
2         AMERICAN SAMOA
3                ANDORRA
4                 ANGOLA
5               ANGUILLA
6    ANTIGUA AND BARBUDA
7              ARGENTINA
8                ARMENIA
9                  ARUBA
Name: Country, dtype: object


In [299]:
%pip install pycountry
%pip install gpdvega
import pycountry

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [300]:
def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df_currency_pct['Country_Code'] = df_currency_pct['Country'].apply(get_country_code)
df_visitors_pct['Country_Code'] = df_visitors_pct['Country'].apply(get_country_code)

# Drop rows where country code couldn't be found
df_currency_pct.dropna(subset=['Country_Code'], inplace=True)
df_visitors_pct.dropna(subset=['Country_Code'], inplace=True)

# Merge on standardized country code instead of name
df_combined = df_currency_pct.merge(df_visitors_pct, on="Country_Code", suffixes=('_currency', '_visitors'))


df_combined

,Country_currency,1995_currency,1996_currency,1997_currency,1998_currency,1999_currency,2000_currency,2001_currency,2002_currency,2003_currency,...,2013_visitors,2014_visitors,2015_visitors,2016_visitors,2017_visitors,2018_visitors,2019_visitors,2020_visitors,2021_visitors,2022_visitors
0,Aruba,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.559082,4.319136,5.347901,-4.039301,5.972696,1.825013,2.846600,-68.067658,51.364366,81.442206
1,Angola,NaN,4555.217420,78.896801,71.508634,610.422376,259.785071,119.687910,97.345542,71.389723,...,23.106061,-8.461538,-0.504202,-32.939189,-34.256927,-16.475096,0.000000,-70.642202,0.000000,103.125000
2,Albania,NaN,12.731106,42.521015,1.141733,-8.592222,4.371274,-0.156276,-2.321024,-13.050786,...,-7.342060,12.807125,12.469371,14.645364,8.065878,15.806956,8.081660,-58.507649,114.033108,32.603269
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.835443,1.576342,0.679749,2.229299,1.582555,2.158979,-1.116715,-36.769885,4.129057,55.416820
4,United Arab Emirates,NaN,0.000000,0.003405,0.037454,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,inf,18.981025,8.186196,4.360103,5.902316,9.483804,-68.024682,58.139535,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Vanuatu,NaN,-0.350097,3.718401,10.049048,1.221401,6.638259,5.571768,-4.207599,-12.219375,...,11.214953,-7.843137,-12.765957,22.508711,-5.375427,5.259994,-26.898915,-67.812500,-100.000000,inf
177,Samoa,NaN,-0.454081,3.947682,15.172285,2.220900,9.067233,5.832546,-2.926417,-11.936897,...,-7.407407,5.600000,5.303030,5.035971,8.219178,8.860759,5.232558,-86.795580,-89.121339,1846.153846
178,South Africa,NaN,18.534558,7.178121,19.972443,10.513208,13.591068,24.054666,22.436116,-28.233270,...,9.556967,1.480654,-3.977977,8.378727,-0.965545,0.193656,-1.379632,-73.733865,-31.456800,175.600601
179,Zambia,NaN,39.783961,8.825027,41.656349,28.245460,30.268811,16.075728,21.813187,7.608703,...,6.519208,3.497268,-1.583949,2.575107,5.543933,6.243806,18.097015,-60.347551,10.358566,-100.000000


In [301]:
import geopandas as gpd
import json
from vega_datasets import data
import gpdvega

In [302]:
countries = alt.topo_feature(data.world_110m.url, 'countries')
background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    "equirectangular"
).properties(
    width=500,
    height=300
)


In [303]:
# Normalize missing values
df_combined.fillna(0, inplace=True)
df_combined['Country_Code']

0      ABW
1      AGO
2      ALB
3      AND
4      ARE
      ... 
176    VUT
177    WSM
178    ZAF
179    ZMB
180    ZWE
Name: Country_Code, Length: 181, dtype: object

In [304]:
country_id_map = pd.read_csv("https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv")
country_id_map = country_id_map[['ISO3166-1-Alpha-3', 'M49']]
country_id_map.columns = ['Country_Code', 'id']

# Merge the country IDs into df_combined
df_combined = df_combined.merge(country_id_map, on="Country_Code", how="left")
df_combined.head()

,Country_currency,1995_currency,1996_currency,1997_currency,1998_currency,1999_currency,2000_currency,2001_currency,2002_currency,2003_currency,...,2014_visitors,2015_visitors,2016_visitors,2017_visitors,2018_visitors,2019_visitors,2020_visitors,2021_visitors,2022_visitors,id
0,Aruba,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.319136,5.347901,-4.039301,5.972696,1.825013,2.846600,-68.067658,51.364366,81.442206,533
1,Angola,0.0,4555.217420,78.896801,71.508634,610.422376,259.785071,119.687910,97.345542,71.389723,...,-8.461538,-0.504202,-32.939189,-34.256927,-16.475096,0.000000,-70.642202,0.000000,103.125000,24
2,Albania,0.0,12.731106,42.521015,1.141733,-8.592222,4.371274,-0.156276,-2.321024,-13.050786,...,12.807125,12.469371,14.645364,8.065878,15.806956,8.081660,-58.507649,114.033108,32.603269,8
3,Andorra,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.576342,0.679749,2.229299,1.582555,2.158979,-1.116715,-36.769885,4.129057,55.416820,20
4,United Arab Emirates,0.0,0.000000,0.003405,0.037454,0.000000,0.000000,0.000000,0.000000,0.000000,...,inf,18.981025,8.186196,4.360103,5.902316,9.483804,-68.024682,58.139535,-100.000000,784


In [305]:
df_combined = df_combined.dropna(subset=['2019_currency', '2019_visitors'])
df_combined = df_combined[df_combined['Country_currency'] != 'Sudan']
df_combined

,Country_currency,1995_currency,1996_currency,1997_currency,1998_currency,1999_currency,2000_currency,2001_currency,2002_currency,2003_currency,...,2014_visitors,2015_visitors,2016_visitors,2017_visitors,2018_visitors,2019_visitors,2020_visitors,2021_visitors,2022_visitors,id
0,Aruba,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.319136,5.347901,-4.039301,5.972696,1.825013,2.846600,-68.067658,51.364366,81.442206,533
1,Angola,0.0,4555.217420,78.896801,71.508634,610.422376,259.785071,119.687910,97.345542,71.389723,...,-8.461538,-0.504202,-32.939189,-34.256927,-16.475096,0.000000,-70.642202,0.000000,103.125000,24
2,Albania,0.0,12.731106,42.521015,1.141733,-8.592222,4.371274,-0.156276,-2.321024,-13.050786,...,12.807125,12.469371,14.645364,8.065878,15.806956,8.081660,-58.507649,114.033108,32.603269,8
3,Andorra,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.576342,0.679749,2.229299,1.582555,2.158979,-1.116715,-36.769885,4.129057,55.416820,20
4,United Arab Emirates,0.0,0.000000,0.003405,0.037454,0.000000,0.000000,0.000000,0.000000,0.000000,...,inf,18.981025,8.186196,4.360103,5.902316,9.483804,-68.024682,58.139535,-100.000000,784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Vanuatu,0.0,-0.350097,3.718401,10.049048,1.221401,6.638259,5.571768,-4.207599,-12.219375,...,-7.843137,-12.765957,22.508711,-5.375427,5.259994,-26.898915,-67.812500,-100.000000,inf,548
177,Samoa,0.0,-0.454081,3.947682,15.172285,2.220900,9.067233,5.832546,-2.926417,-11.936897,...,5.600000,5.303030,5.035971,8.219178,8.860759,5.232558,-86.795580,-89.121339,1846.153846,882
178,South Africa,0.0,18.534558,7.178121,19.972443,10.513208,13.591068,24.054666,22.436116,-28.233270,...,1.480654,-3.977977,8.378727,-0.965545,0.193656,-1.379632,-73.733865,-31.456800,175.600601,710
179,Zambia,0.0,39.783961,8.825027,41.656349,28.245460,30.268811,16.075728,21.813187,7.608703,...,3.497268,-1.583949,2.575107,5.543933,6.243806,18.097015,-60.347551,10.358566,-100.000000,894


In [310]:
base = alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('2022_currency:Q', scale=alt.Scale(scheme='redblue')),
    tooltip=['Country_Code:N', '2022_currency:Q']
).transform_lookup(
    lookup='id', 
    from_=alt.LookupData(df_combined, 'id', ['Country_Code', '2022_currency'])
).project(
    type='naturalEarth1'
).properties(
    width=800,
    height=500,
    title="Currency Percent Changes in 2019"
)

base


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future

alt.Chart(...)

In [212]:
df_combined[['2019_currency', '2019_visitors']] = df_combined[['2019_currency', '2019_visitors']].apply(pd.to_numeric, errors='coerce')


In [282]:
correlation_scatter = alt.Chart(df_combined).mark_circle().encode(
    x=alt.X('2019_currency:Q', title="Currency Change (%)"),
    y=alt.Y('2019_visitors:Q', title="Visitor Change (%)"),
    tooltip=['Country_Code:N', '2022_currency:Q', '2022_visitors:Q']
)

correlation_scatter

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future

alt.Chart(...)

In [214]:
scatter_with_trend = correlation_scatter + correlation_scatter.transform_regression(
    '2019_currency', '2019_visitors'
).mark_line(color="red")

scatter_with_trend

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future

alt.LayerChart(...)

In [292]:
df_merged = df_combined.merge(country_id_map, on="Country_Code", how="left")


map_chart = alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color(
        '2022_currency:Q',
        scale=alt.Scale(
            scheme="redblue"
        ),
        title="Currency Percent Change (%)"),
    tooltip=['Country_Code:N', '2022_currency:Q', '2022_visitors:Q']
).transform_lookup(
    lookup='id', 
    from_=alt.LookupData(df_combined, 'id', ['Country_Code', '2022_currency', '2022_visitors'])
).project(
    type='naturalEarth1'
).properties(
    width=800,
    height=500,
    title="Currency Percent Changes (2019)"
)

map_chart

MergeError: Passing 'suffixes' which cause duplicate columns {'id_x'} is not allowed.

In [216]:
# reshape df
df_melted = df_combined.melt(id_vars=['Country_Code'], 
    var_name='Metric', 
    value_name='Value')

# Extract Year and Metric Type
df_melted['Year'] = df_melted['Metric'].str.extract(r'(\d{4})')
df_melted['Type'] = df_melted['Metric'].str.extract(r'_(\w+)$')

# Pivot table so that we have separate columns for Currency & Visitors
df_melted = df_melted.pivot_table(index=['Country_Code', 'Year'], 
    columns='Type', 
    values='Value').reset_index()

# Rename columns for clarity
df_melted.columns = ['Country_Code', 'Year', 'Currency', 'Visitors']

# Convert year to integer for selection
df_melted['Year'] = df_melted['Year'].astype(int)
df_melted.head()

,Country_Code,Year,Currency,Visitors
0,ABW,1995,0.0,0.0
1,ABW,1996,0.0,4.934211
2,ABW,1997,0.0,-1.044932
3,ABW,1998,0.0,-4.329461
4,ABW,1999,0.0,7.284768


In [317]:

select_year = alt.selection_point(
    name="Year",
    fields=["Year"],
    bind=alt.binding_range(min=1995, max=2022, step=1, name="Year"),
    value=2022,
)

scatter_chart = alt.Chart(df_melted).mark_circle(size=80).encode(
    x=alt.X('Currency:Q', title="Currency Change (%)"),
    y=alt.Y('Visitors:Q', title="Visitor Change (%)"),
    color=alt.Color(
        'Currency:Q',
        scale=alt.Scale(
            scheme="redblue",
            domain=[-20,0,80]
        ),
        title="Currency Percent Change (%)"),
        tooltip=[
        alt.Tooltip('Country_Code:N', title="Country"),
        alt.Tooltip('Currency:Q', title="Currency Change (%)"),
        alt.Tooltip('Visitors:Q', title="Visitor Change (%)"),
        alt.Tooltip('Year:O', title="Year Selected")
    ]
).add_selection(
    select_year
).transform_filter(
    select_year
).properties(
    title="Scatter Plot of Currency Changes vs. Visitor Changes (Yearly)",
    width=700,
    height=500
).interactive()

horizontal_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    strokeDash=[5, 5],
    color='black'
).encode(
    y='y:Q'
)

vertical_line = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(
    strokeDash=[5, 5],
    color='black'
).encode(
    x='x:Q'
)

scatter_chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.Chart(...)

In [239]:
scatter_chart.save("scatter_currency_tourism.html")

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [284]:
df_merged = df_combined.merge(country_id_map, on="Country_Code", how="left")


map_chart = alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color(
        '2022_visitors:Q',
        scale=alt.Scale(
            scheme="redblue"
        ),
        title="Visitor Percent Change (%)"),
    tooltip=['Country_Code:N', '2022_currency:Q', '2022_visitors:Q']
).transform_lookup(
    lookup='id', 
    from_=alt.LookupData(df_combined, 'id', ['Country_Code', '2022_currency', '2022_visitors'])
).project(
    type='naturalEarth1'
).properties(
    width=800,
    height=500,
    title="International Visitor Percent Change (2022)"
)

map_chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future

alt.Chart(...)

In [318]:

scatter_plus_lines = scatter_chart + horizontal_line + vertical_line
scatter_plus_lines

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/data.py:257: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future

alt.LayerChart(...)

In [287]:
scatter_plus_lines.save("scatter_with_lines_currency_tourism.html")

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
